In [ ]:
import torch
import os
import json
from matplotlib import pyplot as plt
import sys
sys.path.append('/nfs/home/clruben/workspace/nst/')

from dataloaders import Dataloader
from models.maegan.network import MaeGanLM

torch.set_float32_matmul_precision('medium')

In [ ]:
CONFIG_PATH = "/nfs/home/clruben/workspace/nst/models/maegan/config.json"

with open(CONFIG_PATH, encoding="utf-8") as file:
    config = json.load(file)

In [ ]:
DATA_PATH = "/nfs/home/clruben/workspace/nst/data/"

test_loader = Dataloader(
    DATA_PATH,
    'test',
    batch_size=config["batch_size"]
)

In [ ]:
### sample inference code: 

checkpoint = "/nfs/home/clruben/workspace/nst/models/maegan/checkpoints/model_checkpoint-v1.ckpt"
checkpoint_module = MaeGanLM.load_from_checkpoint(checkpoint, config=config)

# choose your trained nn.Module
model = checkpoint_module.model
model.eval() # set model to eval mode (for dropout and batchnorm)

In [ ]:
# embed 4 fake images!
images, data_type = test_loader.__next__()
outputs = model.forward_mae(images.cuda(), data_type)

In [ ]:
is_nan = torch.stack([torch.isnan(p).any() for p in model.parameters()]).any()
print(is_nan)

In [ ]:
outputs[0].shape

In [ ]:
images = model.unpatchify(outputs[0].detach().cpu()).numpy()

In [ ]:
plt.imshow(images[0][0])